In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/MyDrive/AFCS

In [ ]:
# !pip install pmdarima

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import pearsonr
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace import exponential_smoothing
from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.api import STLForecast

In [ ]:
from utils import *

In [ ]:
data = Data()

In [ ]:
data.sales_train

In [ ]:
total_sales = data.total_sales

plt.figure(figsize=(14, 6))
plt.plot(data.total_sales.index, data.total_sales.values, label='Total Sales', color='blue')
plt.title('Total Daily Unit Sales (Aggregated for All Products)')
plt.xlabel('Date')
plt.ylabel('Total Unit Sales')
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 1913, 100))
plt.show()

In [ ]:


data.merged_data

In [ ]:
# now we can show sales with actual dates, day of week etc.


plt.figure(figsize=(12, 6))
plt.plot(data.daily_sales.index, data.daily_sales.values, label='Aggregated Sales', color='blue')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

# Perform seasonal decomposition for both period 7 and period 365
mstl = MSTL(data.daily_sales, periods=[7, 365])
res = mstl.fit()
res.seasonal.head()
ax = res.plot()

# Perform seasonal decomposition
#result = seasonal_decompose(daily_sales, model='additive', period=365)

# Plot the decomposed components
#plt.figure(figsize=(12, 8))

#plt.subplot(4, 1, 1)
#plt.plot(daily_sales, label='Original')
#plt.title('Seasonal Decomposition of Aggregated Sales over Time')
#plt.legend(loc='upper left')

#plt.subplot(4, 1, 2)
#plt.plot(result.trend, label='Trend')
#plt.legend(loc='upper left')

#plt.subplot(4, 1, 3)
#plt.plot(result.seasonal, label='Seasonal')
#plt.legend(loc='upper left')

#plt.subplot(4, 1, 4)
#plt.plot(result.resid, label='Residuals')
#plt.legend(loc='upper left')

#plt.tight_layout()
#plt.show()

In [ ]:
# Calculate the ACF
acf_result = sm.tsa.acf(data.daily_sales, nlags=1500)

# Plot the ACF
plt.figure(figsize=(12, 6))
plt.stem(range(len(acf_result)), acf_result, markerfmt='bo', linefmt='b-', basefmt='r-')
plt.title('Autocorrelation Function (ACF) for Aggregated Sales')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data.daily_sales[:364].index, data.daily_sales[:364].values, label='Aggregated Sales', color='blue')
plt.title('Total Daily Unit Sales for First Train Year (Aggregated for All Products)')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

# Perform seasonal decomposition
result = seasonal_decompose(data.daily_sales[:364], model='additive', period=7)

# Plot the decomposed components
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(data.daily_sales[:364], label='Original')
plt.legend(loc='upper left')

plt.subplot(4, 1, 2)
plt.plot(result.trend, label='Trend')
plt.legend(loc='upper left')

plt.subplot(4, 1, 3)
plt.plot(result.seasonal, label='Seasonal')
plt.legend(loc='upper left')

plt.subplot(4, 1, 4)
plt.plot(result.resid, label='Residuals')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Seems like sales are very low on christmas day

print(data.daily_sales[data.daily_sales < 10].index)

print(data.daily_sales[data.daily_sales > 3000].index)

In [ ]:



plt.figure(figsize=(10, 6))
plt.bar(data.weekly_avg_sales.index, data.weekly_avg_sales.values, color='green', alpha=0.7)
plt.title('Weekly Average Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Sales')
plt.grid(axis='y')
plt.show()

In [ ]:


plt.figure(figsize=(14, 6))
plt.plot(data.mean_prices.index, data.mean_prices.values, label='Mean Sell Prices', color='green')
plt.title('Mean Sell Prices per Week')
plt.xlabel('Date')
plt.ylabel('Mean Sell Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Calculate the correlation coefficient
correlation_coefficient, _ = pearsonr(data.sales_sellprice_merge['sales'], data.sales_sellprice_merge['sell_price'])

# Plotting aggregated sales
plt.figure(figsize=(12, 6))
plt.scatter(data.sales_sellprice_merge['sell_price'], data.sales_sellprice_merge['sales'])
plt.title(f'Scatter Plot of Aggregated Sales and Mean Sell Price\nCorrelation: {correlation_coefficient:.2f}')
plt.xlabel('Mean Sell Price')
plt.ylabel('Aggregated Sales')
plt.show()

# Display the correlation coefficient
print(f'Correlation Coefficient: {correlation_coefficient:.2f}')

In [ ]:
from model_prophet import *


In [ ]:
model = ProphetModel(data.merged_data)


In [ ]:
all_forecasts = model.generate_forecasts()

In [ ]:
# model.df_all_forecasts
# model.save_forecasts_to_csv('submission_prophet_model.csv')

In [ ]:
rmse = model.compute_rmse(data.sales_test)

print(f'RMSE: {rmse}')

# RMSE without holidays: 3.3133914271517413
# RMSE with holidays: 3.313470007778738

In [ ]:
# In this cell, we will use a Prophet model on aggregated sales.
# We use weekly and yearly seasonality and include our events/holidays.

# Prepare the dataframe for Prophet
prophet_df = pd.DataFrame({'ds': data.daily_sales.index, 'y': data.daily_sales.values})
prophet_df['floor'] = 0

# Instantiate Prophet model with yearly and weekly seasonality
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)

# Fit the model
model.fit(prophet_df)

# Create a dataframe for future dates (next 28 days)
future = model.make_future_dataframe(periods=28)

# Generate forecasts
forecast = model.predict(future)

# Round to nearest int
forecast['yhat'] = forecast['yhat'].round()

In [ ]:
#forecast['yhat']
#model.plot_components(forecast)

In [ ]:
sales_test_long = pd.melt(data.sales_test, id_vars=['id'],
                           var_name='day', value_name='sales')
sales_test_long['day'] = pd.to_datetime('2011-01-29') + pd.to_timedelta(sales_test_long['day'].str[2:].astype(int) - 1, unit='D')
data.calendar['date'] = pd.to_datetime(data.calendar['date'])

merged_test_data = pd.merge(sales_test_long, data.calendar, left_on='day', right_on='date', how='left')

daily_sales_test = merged_test_data.groupby('day')['sales'].sum()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data.daily_sales.index, data.daily_sales.values, color='blue')
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.plot(data.daily_sales.index, forecast['yhat'][:1913], label='Aggregated Sales (Prophet)', color='red')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
arima_df = pd.DataFrame({'ds': data.daily_sales.index, 'y': data.daily_sales.values})

autoarima_model = auto_arima(arima_df['y'], seasonal=True, trace=True)
autoarima_model.summary()
autoarima_forecast = autoarima_model.predict(n_periods=28)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.plot(daily_sales_test.index, autoarima_forecast.values, label='Aggregated Sales (ARIMA)', color='red')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
!pip install tbats

In [ ]:
# TBATS model on aggregated sales data

tbats_df = pd.DataFrame({'ds': daily_sales.index, 'y': daily_sales.values})

from tbats import TBATS
tbats_model = TBATS(seasonal_periods=(7, 365))
tbats_fit = tbats_model.fit(tbats_df['y'])
tbats_forecast = tbats_fit.forecast(steps=28)


In [ ]:
# Reading model parameters
print(tbats_fit.params.alpha)
print(tbats_fit.params.beta)
print(tbats_fit.params.x0)
print(tbats_fit.params.components.use_box_cox)
print(tbats_fit.params.components.seasonal_harmonics)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.plot(daily_sales_test.index, autoarima_forecast.values, label='Aggregated Sales (ARIMA)', color='red')
plt.plot(daily_sales_test.index, tbats_forecast, label='Aggregated Sales (TBATS)', color='yellow')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Get mean distribution of sales for each product

merged_data['total_sales'] = merged_data.groupby('day')['sales'].transform('sum')
merged_data['percentage_sales'] = (merged_data['sales'] / merged_data['total_sales'])
mean_percentage_by_product = merged_data.groupby('id')['percentage_sales'].mean().reset_index()
print(mean_percentage_by_product)

In [ ]:
estimated_sales = (mean_percentage_by_product['percentage_sales'][:, np.newaxis] * tbats_forecast).round()

#print(sales_test.iloc[:, 1:])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(sales_test.iloc[:, 1:], estimated_sales))

print(f'RMSE: {rmse}')